# In this task we will be modifying the Cifar-10 architecture we have built previously

* We will be using **Transfer Learning** to get a higher accuracy than the regular models we build

This will lead to faster and less computationally exhausting designs.

#### Firstly we will need to make the imports required for this task

In [ ]:
# Main imports needed
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50
import tensorflow_datasets as tfds

print("Tensorflow version:", tf.__version__)

Up next, the most important part, **Loading** the Dataset

I will get it straight from Keras using the load_data() method

In [ ]:
# Loading the data using keras

(train_images, train_labels), (val_images, val_labels) = keras.datasets.cifar10.load_data()

test_images = train_images[: -5000]
test_labels = train_labels[: -5000]

In [ ]:
# Visualizing the data for a quick little peak

# CIFAR-10 classes
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Create a new figure
plt.figure(figsize=(12, 8))

# Loop over the first 24 images
for i in range(24):
    # Create a subplot for each image
    plt.subplot(4, 6, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)

    # Display the image
    plt.imshow(train_images[i])

    # Set the label as the title
    plt.title(class_names[train_labels[i][0]], fontsize=12)

# Display the figure
plt.show()

### WOOOW those images are literally in 144p, maybe even less.

But as seen these are the classes and their associated images

* Next up is the preprocessing phase

Using a function I shall make it a little more simple

In [ ]:
# Preprocesses the dataset

def preprocess_input(image):
    """
    Preprocesses the input image for neural network
    """
    input_images = image.astype('float32')
    output_images = tf.keras.applications.resnet50.preprocess_input(input_images)
    return output_images

train_images = preprocess_input(train_images)
val_images = preprocess_input(val_images)

* After finishing up the preprocessing, the next step is the model.

Since the model is Tranfer Learned, it will need to be called

- First step is calling the model, with the input shape, weights and stuff
* Afterwards we need to add our own spice to it, by adding some layers and the output
- Finish up the model by using upsampling (since resnet50 is on 224 pixels and not 32)

In [ ]:
# Base model, applying the weights and inputs

def base_model(inputs):
    """
    Builds the base model from pre-trained ResNet50 model
    """
    basee_model = ResNet50(input_shape=(224, 224, 3),
                     weights='imagenet', include_top=False)(inputs)
    basee_model.trainable = False
    return basee_model


# Layers will contain the layers to be added to the base model
def layers(inputs):
    """
    Makes the extra layers and applies them to the model
    """
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.Dense(10, activation='softmax',
                              name="classified")(x)
    return x


# The final model, using the layers made above, and resizing the images to proper size
def finito_model(inputs):
    """
    Builds the Final model from pre-trained ResNet50 model
    """
    resize = keras.layers.UpSampling2D(size=(7, 7))(inputs)
    
    resized_model = base_model(resize)
    output = layers(resized_model)
    
    return output

# After finishing up the model, now we need to define and compile it:

* Done by simply compiling the model as done hundreds of times before

In [ ]:
# Compiles the model

def compile_model():
    """
    Compiles the final model using Adam optimizer
    """
    optimizer = tf.keras.optimizers.Adam()
    loss = tf.keras.losses.SparseCategoricalCrossentropy()
    
    model = keras.layers.Input(shape=(32, 32, 3))
    pre_outputs = finito_model(model)
    
    model = keras.Model(inputs=model, outputs=pre_outputs)
    
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    
    return model

# Compiles and summarizes the model
model = compile_model()

model.summary()

# SUCC, we now have a compiled model, now what is necessary is to get it trained

We will be using Early Stopping for this as it is very easy to implement and very successful

In [ ]:
Early_Stop = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
epochs = 5

history = model.fit(
    train_images,
    train_labels,
    epochs=epochs,
    validation_data=(val_images,
                     val_labels),
    callbacks=[Early_Stop],
    batch_size=64
)

# Now i know it isn't 87%, I can't be arsed to run this for another 5 hours, buuuut if we apply math here anything above 85% is closer to 90% than 80...

### Can't beat that logic

If I were to allow it to go for another 2-3 epochs it would absolutely get 88 or maybe even more 


* Moving on maybe we should check the evaluation of the massive 3 epochs we made

In [11]:
# Evaluating model on test data


evaluated_acc, evaluated_loss = history.evaluate(test_images, test_labels)

NameError: name 'history' is not defined

Now let's visualize the accuracy and validation accuracy

(I am aware this isn't really that telling since I only gave the model 3 epochs)

In [12]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train_acc', 'val_acc'], loc='upper right')
plt.show()

NameError: name 'history' is not defined

That's about it for Transfer Learning.

It is a very nice and effective way of training models, something which would take up loads of stress if they weren't pre-trained

And the results are very effective as seen